In [ ]:
!pip install numpy==1.26.4 scipy==1.10.1 --force-reinstall --upgrade

In [ ]:
!pip install pandas==2.2.2 gensim==4.3.2 sentence-transformers==2.2.2 huggingface-hub==0.23.0

In [ ]:
!pip install -q spacy==3.7.2 nltk wordcloud seaborn scikit-learn matplotlib

In [ ]:
!pip install -q cqdm PyMuPDF pdfminer.six

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import nltk  # Correct spelling
import fitz  # PyMuPDF works via the 'fitz' module

import gensim.downloader as api
from tqdm import tqdm
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, roc_curve, auc  # Corrected names


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from wordcloud import WordCloud
from sentence_transformers import SentenceTransformer  # Fixed import path
from collections import Counter

# If using Google Colab
from google.colab import files

# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# Load stopwords
stop_words = set(stopwords.words('english'))

# Load SpaCy model
import spacy
nlp = spacy.load("en_core_web_sm")  # Corrected assignment and removed stray bracket


In [ ]:
from google.colab import files
import fitz  # PyMuPDF
import os

# Upload files
uploaded = files.upload()

# Text extraction function
def extract_text(file_path):
    ext = os.path.splitext(file_path)[1].lower()

    if ext == '.pdf':
        doc = fitz.open(file_path)
        return ''.join([page.get_text() for page in doc])
    else:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()

# Lists to store data
resumes = []
names = []

# Process each uploaded file
for file in uploaded:
    text = extract_text(file)
    resumes.append(text)
    names.append(file)
df = pd.DataFrame({'filename': names, 'raw_text': resumes})

In [ ]:
def clean_text(txt):
    txt = txt.lower()
    txt = re.sub(r'\s+', ' ', txt)
    txt = re.sub(r'[^\w\s]', '', txt)
    return txt
def extract_entities(text):
    doc = nlp(text)
    entities = {'name': None, 'email': None, 'phone': None}
    for ent in doc.ents:
        if ent.label_ == 'PERSON' and not entities['name']:
            entities['name'] = ent.text
    email = re.findall(r'\S+@\S+', text)
    entities['email'] = email[0] if email else None
    phone = re.findall(r'\+?\d[\d-]{8,13}', text)
    entities['phone'] = phone[0] if phone else None
    return entities

df['cleaned'] = df['raw_text'].apply(clean_text)
entities = df['raw_text'].apply(extract_entities).apply(pd.Series)
df = pd.concat([df, entities], axis=1)


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')


In [ ]:
base_skills = [
    'python', 'java', 'sql', 'r', 'excel', 'communication', 'data analysis', 'nlp', 'ai',
    'tensorflow', 'keras', 'pandas', 'numpy', 'flask', 'django', 'cloud', 'aws', 'linux'
]
w2v = api.load("glove-wiki-gigaword-100")
expanded_skills = set(base_skills)
for skill in base_skills:
    try:
        similar = w2v.most_similar(skill, topn=3)
        for word, score in similar:
            expanded_skills.add(word)
    except KeyError:
        continue

In [ ]:
def extract_skills(text):
    tokens = [t.lower() for t in word_tokenize(text) if t.isalpha()]
    tokens = [t for t in tokens if t not in stop_words]
    found = set(t for t in tokens if t in expanded_skills)
    return list(found)
df['skills'] = df['cleaned'].apply(extract_skills)

In [ ]:
jd_files_data = uploaded
jd_texts = []
jd_names = []
for name, content in jd_files_data.items():
    with open(name, 'wb') as f:
        f.write(content)
    text = extract_text(name)
    jd_texts.append(clean_text(text))
    jd_names.append(name)
uploaded=()
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
resume_embeddings = model.encode(df['cleaned'].tolist(), show_progress_bar=True)
jd_embeddings = model.encode(jd_texts, show_progress_bar=True)
similarity_matrix = cosine_similarity(resume_embeddings, jd_embeddings)
best_match_idx = np.argmax(similarity_matrix, axis=1)
best_scores = np.max(similarity_matrix, axis=1)
df['best_match_jd'] = [jd_names[i] for i in best_match_idx]
df['similarity_score'] = best_scores


In [ ]:
top_n = 5
for jd_name in jd_names:
    print(f"\nTop {top_n} Matches for JD: {jd_name}")

    jd_matches = df[df['best_match_jd'] == jd_name] \
                   .sort_values(by='similarity_score', ascending=False) \
                   .head(top_n)

    display(jd_matches[['filename', 'name', 'email', 'skills', 'similarity_score']])


In [ ]:
skill_freq = {}
for skills in df['skills']:
    for s in skills:
        skill_freq[s] = skill_freq.get(s, 0) + 1
wc = WordCloud(width=1000, height=500, background_color="white")
wc.generate_from_frequencies(skill_freq)
plt.figure(figsize=(12, 6))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.title("All Resume Skill Frequencies")
plt.show()
sns.histplot(df['similarity_score'], kde=True, color="teal")
plt.title("Resume vs JD Similarity", )
plt.xlabel("Cosine Similarity",)
plt.show()

In [ ]:
from sklearn.metrics import classification_report
df['match'] = df['similarity_score'].apply(lambda x: 1 if x > 0.65 else 0)
true = [1 if i < len(df)//2 else 0 for i in range(len(df))]
pred = df['match'].tolist()
precision = precision_score(true, pred)
recall = recall_score(true, pred)
f1 = f1_score(true, pred)
classi = classification_report(true, pred)
print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}")
print(classi)
fpr, tpr, _ = roc_curve(true, df['similarity_score'])
roc_auc = auc(fpr, tpr)

plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve for Resume-JD Matching")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
n_clusters = 2
if len(resume_embeddings) >= n_clusters:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10) # Add n_init
    clusters = kmeans.fit_predict(resume_embeddings)
    df['cluster'] = clusters
    pca = PCA(n_components=2)
    pca_components = pca.fit_transform(resume_embeddings)

    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=pca_components[:, 0], y=pca_components[:, 1], hue=clusters, palette="Set2")
    plt.title("Resume Skill Clustering via T5 + PCA")
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.show()
else:
    print(f"Cannot perform KMeans with {n_clusters} clusters. Need {n_clusters} resumes, but found {len(resume_embeddings)}.")


In [ ]:
def jd_skill_words(text):
    tokens = [t.lower() for t in word_tokenize(text) if t.isalpha()]
    return [t for t in tokens if t not in stop_words and t in expanded_skills]

for i, jd_text in enumerate(jd_texts):
    jd_skills = jd_skill_words(jd_text)
    jd_skill_freq = Counter(jd_skills)

    resume_skills = []
    for skills in df[df['best_match_jd'] == jd_names[i]]['skills']:
        resume_skills.extend(skills)

    resume_skill_freq = Counter(resume_skills)
    common_skills = list(set(jd_skill_freq.keys()) & set(resume_skill_freq.keys()))
    bar_vals = [resume_skill_freq[s] for s in common_skills]

    plt.figure(figsize=(10, 5))
    sns.barplot(x=common_skills, y=bar_vals, palette="viridis")
    plt.title(f"Skill Overlap Frequency (JD: {jd_names[i]})")
    plt.ylabel("Count in Matched Resumes")
    plt.xticks(rotation=45)
    plt.show()